# Demo 3: Random Forest and DataStax Analytics
------
<img src="images/drinkWine.jpeg" width="300" height="500">


#### Dataset: https://archive.ics.uci.edu/ml/datasets/Wine+Quality

## What are we trying to learn from this dataset? 

# QUESTION: Can Random Forest be used to classify a wine’s rating score by its attributes?

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt

In [2]:
import pandas
import cassandra
import pyspark
import re
import os
import random
from random import randint, randrange
import matplotlib.pyplot as plt
from IPython.display import display, Markdown
from pyspark.sql import SparkSession
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.feature import IndexToString, StringIndexer, VectorAssembler
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
%store -r astraUsername astraPassword astraSecureConnect astraKeyspace

#### Helper function to have nicer formatting of Spark DataFrames

In [3]:
#Helper for pretty formatting for Spark DataFrames
def showDF(df, limitRows =  5, truncate = True):
    if(truncate):
        pandas.set_option('display.max_colwidth', 50)
    else:
        pandas.set_option('display.max_colwidth', None)
    pandas.set_option('display.max_rows', limitRows)
    display(df.limit(limitRows).toPandas())
    pandas.reset_option('display.max_rows')

## Creating Tables and Loading Tables

### Connect to Cassandra

In [4]:
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider

cloud_config = {
    'secure_connect_bundle': '/tmp/'+astraSecureConnect
}
auth_provider = PlainTextAuthProvider(username=astraUsername, password=astraPassword)
cluster = Cluster(cloud=cloud_config, auth_provider=auth_provider)
session = cluster.connect()

### Set keyspace 

In [5]:
session.set_keyspace(astraKeyspace)

### Create table called `wines`. Our PRIMARY will be a unique key (wineid) we generate for each row.  This will have two datasets "white" and "red"

In [ ]:
query = "CREATE TABLE IF NOT EXISTS wines \
                                   (wineid int, fixedAcidity float, volatileAcidity float, citricAcid float, sugar float, \
                                   chlorides float, freeSulfur float, totalSulfur float, density float, ph float, \
                                   sulphates float, alcohol float, quality float, \
                                   PRIMARY KEY (wineid))"
session.execute(query)

### What do these of these 12 columns represent: 

* **Fixed acidity**
* **Volatile acidity**
* **Citric Acid**
* **Residual Sugar** 
* **Chlorides**
* **Free sulfur dioxide**     
* **Total sulfur dioxide**
* **Density** 
* **pH**
* **Sulphates**
* **Alcohol**
* **Quality**

### Load 2 Wine Dataset -- White and Red
<img src="images/whiteAndRed.jpeg" width="300" height="300">

In [ ]:
#download file to local (working on better way)
from google.cloud import storage
storage_client = storage.Client()
bucket = storage_client.get_bucket('andygoade-dev')

#download file for red wines
blob = storage.Blob('notebooks/jupyter/data/winequality-red.csv', bucket)
blob.download_to_filename('/tmp/winequality-red.csv')

#download file for white wines
blob = storage.Blob('notebooks/jupyter/data/winequality-white.csv', bucket)
blob.download_to_filename('/tmp/winequality-white.csv')

### Load Wine datasets from CSV file (winequality-red.csv winequality-white.csv)
* No clean up was requried! How nice :)

#### Insert all the Wine Data into the DSE table `wines`

In [ ]:
fileName = '/tmp/winequality-red.csv'
input_file = open(fileName, 'r')
i = 1
for line in input_file:
    wineid = i
    row = line.split(';')
        
    query = "INSERT INTO wines (wineid, fixedAcidity, volatileAcidity, citricAcid, sugar, \
                               chlorides, freeSulfur, totalSulfur, density, ph, \
                               sulphates, alcohol, quality)"
    query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
    session.execute(query, (wineid, float(row[0]), float(row[1]), float(row[2]), float(row[3]), float(row[4]), float(row[5]), float(row[6]), float(row[7]), float(row[8]), float(row[9]), float(row[10]), float(row[11])))
    i = i + 1

fileName = '/tmp/winequality-white.csv'
input_file = open(fileName, 'r')

for line in input_file:
    wineid = i
    row = line.split(';')
        
    query = "INSERT INTO wines (wineid, fixedAcidity, volatileAcidity, citricAcid, sugar, \
                               chlorides, freeSulfur, totalSulfur, density, ph, \
                               sulphates, alcohol, quality)"
    query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
    session.execute(query, (wineid, float(row[0]), float(row[1]), float(row[2]), float(row[3]), float(row[4]), float(row[5]), float(row[6]), float(row[7]), float(row[8]), float(row[9]), float(row[10]), float(row[11])))
    i = i + 1
    

## Machine Learning with Apache Cassandra and Apache Spark
<img src="images/sparklogo.png" width="150" height="200">

#### Create a spark session that is connected to the database. From there load each table into a Spark Dataframe and take a count of the number of rows in each.

In [6]:
spark = SparkSession \
    .builder \
    .appName('demo3') \
    .master("local") \
    .config( \
        "spark.cassandra.connection.config.cloud.path", \
        "file:/tmp/"+astraSecureConnect) \
    .config("spark.cassandra.auth.username", astraUsername) \
    .config("spark.cassandra.auth.password", astraPassword) \
    .getOrCreate()

wineDF = spark.read.format("org.apache.spark.sql.cassandra").options(table="wines", keyspace=astraKeyspace).load()

print ("Table Wine Row Count: ")
print (wineDF.count())

Table Wine Row Count: 
6497


In [7]:
showDF(wineDF)

,wineid,alcohol,chlorides,citricacid,density,fixedacidity,freesulfur,ph,quality,sugar,sulphates,totalsulfur,volatileacidity
0,2370,9.8,0.045,0.31,0.99420,6.8,29.0,3.41,7.0,5.20,0.46,120.0,0.120
1,4789,10.2,0.035,0.33,0.99196,6.6,42.0,3.15,5.0,1.80,0.36,148.0,0.190
2,2900,10.2,0.045,0.38,0.99500,9.0,52.0,2.93,6.0,5.90,0.35,159.0,0.245
3,3614,11.8,0.026,0.30,0.99300,9.0,14.0,2.94,5.0,8.10,0.36,71.0,0.550
4,2069,9.6,0.047,0.53,0.99920,7.2,59.0,3.09,5.0,18.15,0.52,182.0,0.290


#### Let's filter out only wines that have been rated 6.0 or higher and create a new dataframe with that information 

In [8]:
wine6DF = wineDF.filter("quality > 5")
showDF(wine6DF)

,wineid,alcohol,chlorides,citricacid,density,fixedacidity,freesulfur,ph,quality,sugar,sulphates,totalsulfur,volatileacidity
0,2370,9.8,0.045,0.31,0.99420,6.8,29.0,3.41,7.0,5.2,0.46,120.0,0.120
1,2900,10.2,0.045,0.38,0.99500,9.0,52.0,2.93,6.0,5.9,0.35,159.0,0.245
2,5384,10.1,0.064,0.27,0.99776,6.5,29.0,3.20,6.0,17.4,0.49,140.0,0.250
3,3913,12.6,0.029,0.30,0.99061,6.8,15.0,3.33,7.0,3.3,0.63,80.0,0.320
4,4425,10.5,0.048,0.22,0.99256,6.3,41.0,3.16,6.0,3.3,0.50,161.0,0.250


#### Create Vector with all elements of the wine 

In [9]:
assembler = VectorAssembler(
    inputCols=['alcohol', 'chlorides', 'citricacid', 'density', 'fixedacidity', 'ph', 'freesulfur', 'sugar', 'sulphates', 'totalsulfur', 'volatileacidity'],
    outputCol='features')

trainingData = assembler.transform(wine6DF)

labelIndexer = StringIndexer(inputCol="quality", outputCol="label", handleInvalid='keep')
trainingData1 = labelIndexer.fit(trainingData).transform(trainingData)

showDF(trainingData1)
print(trainingData1.count())

,wineid,alcohol,chlorides,citricacid,density,fixedacidity,freesulfur,ph,quality,sugar,sulphates,totalsulfur,volatileacidity,features,label
0,2370,9.8,0.045,0.31,0.99420,6.8,29.0,3.41,7.0,5.2,0.46,120.0,0.120,"[9.800000190734863, 0.04500000178813934, 0.310...",1.0
1,2900,10.2,0.045,0.38,0.99500,9.0,52.0,2.93,6.0,5.9,0.35,159.0,0.245,"[10.199999809265137, 0.04500000178813934, 0.37...",0.0
2,5384,10.1,0.064,0.27,0.99776,6.5,29.0,3.20,6.0,17.4,0.49,140.0,0.250,"[10.100000381469727, 0.06400000303983688, 0.27...",0.0
3,3913,12.6,0.029,0.30,0.99061,6.8,15.0,3.33,7.0,3.3,0.63,80.0,0.320,"[12.600000381469727, 0.028999999165534973, 0.3...",1.0
4,4425,10.5,0.048,0.22,0.99256,6.3,41.0,3.16,6.0,3.3,0.50,161.0,0.250,"[10.5, 0.04800000041723251, 0.2199999988079071...",0.0


4113


### We will be training a model with Random Forest, and because of this we need to split up our dataset in to a training and test set. Will split 80/20. 

In [10]:
# Split the data into train and test
splits = trainingData1.randomSplit([0.8, 0.2], 1234)
train = splits[0]
test = splits[1]

print ("Train Dataframe Row Count: ")
print (train.count())
print ("Test Datafram Row Count: ")
print (test.count())

Train Dataframe Row Count: 
3354
Test Dataframe Row Count: 
759


In [11]:
rf = RandomForestClassifier(labelCol="label", featuresCol="features", numTrees=10)

model = rf.fit(train)

predictions = model.transform(test)
#predictions.show()
print (predictions.count())
showDF(predictions)

759


,wineid,alcohol,chlorides,citricacid,density,fixedacidity,freesulfur,ph,quality,sugar,sulphates,totalsulfur,volatileacidity,features,label,rawPrediction,probability,prediction
0,4,9.8,0.075,0.56,0.9980,11.2,17.0,3.16,6.0,1.9,0.58,60.0,0.28,"[9.800000190734863, 0.07500000298023224, 0.560...",0.0,"[8.405459792494058, 1.4182923809713734, 0.1720...","[0.8405459792494057, 0.14182923809713732, 0.01...",0.0
1,9,9.5,0.073,0.02,0.9968,7.8,9.0,3.36,7.0,2.0,0.57,18.0,0.58,"[9.5, 0.0729999989271164, 0.019999999552965164...",1.0,"[8.809031495474525, 1.0739204229104546, 0.1117...","[0.8809031495474523, 0.10739204229104544, 0.01...",0.0
2,21,9.4,0.077,0.48,0.9968,8.9,29.0,3.39,6.0,1.8,0.53,60.0,0.22,"[9.399999618530273, 0.07699999958276749, 0.479...",0.0,"[8.711781410812819, 1.1582898951886882, 0.1245...","[0.8711781410812817, 0.1158289895188688, 0.012...",0.0
3,185,9.3,0.079,0.21,0.9970,6.7,8.0,3.52,6.0,1.9,0.58,62.0,0.62,"[9.300000190734863, 0.07900000363588333, 0.209...",0.0,"[8.809031495474525, 1.0739204229104546, 0.1117...","[0.8809031495474523, 0.10739204229104544, 0.01...",0.0
4,277,9.4,0.077,0.04,0.9987,6.9,7.0,3.69,6.0,3.0,0.91,27.0,0.54,"[9.399999618530273, 0.07699999958276749, 0.039...",0.0,"[8.74241598001976, 1.1555980951051685, 0.09782...","[0.8742415980019759, 0.11555980951051684, 0.00...",0.0


In [12]:
showDF(predictions.select("quality", "label", "prediction", "probability"))

,quality,label,prediction,probability
0,6.0,0.0,0.0,"[0.8405459792494057, 0.14182923809713732, 0.01..."
1,7.0,1.0,0.0,"[0.8809031495474523, 0.10739204229104544, 0.01..."
2,6.0,0.0,0.0,"[0.8711781410812817, 0.1158289895188688, 0.012..."
3,6.0,0.0,0.0,"[0.8809031495474523, 0.10739204229104544, 0.01..."
4,6.0,0.0,0.0,"[0.8742415980019759, 0.11555980951051684, 0.00..."


### We can now use the MutliclassClassifciationEvaluator to evalute the accurancy of our predictions. 

In [13]:
# compute accuracy on the test set
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction",
                                              metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test set accuracy = " + str(accuracy))

Test set accuracy = 0.6982872200263505


In [14]:
session.execute("""drop table wines""")

In [15]:
spark.stop()